In [1]:
import numpy as np
import pandas as pd
import requests
import torch
import wikipedia
from scipy.spatial import distance
from transformers import AutoTokenizer, BertModel
from tqdm.notebook import tqdm, trange
from wikipedia import DisambiguationError
tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")

In [2]:
dataset = pd.read_csv("dataset_with_embeds.csv", sep=";")
dataset["embeds"] = dataset["embeds"].apply(lambda x: np.array(list(map(float, x[1:-1].split(",")))))
dataset.head()

,text,link,artist,title,embeds
0,"""Ah shit, here we go again"" Снова летний трип ...",https://pesnihi.com/lyrics/a/alphavite/kubok-m...,"Alphavite, Hallowen, KnownAim, MIREKU DJIMA, м...",КУБОК МЦ (7),"[0.4156811535358429, -0.510965883731842, 0.116..."
1,"""S1"" (feat. РАМШ) — Да, да — Алё, выйдешь? Всп...",https://pesnihi.com/lyrics/1/104/s1.html,"104, Saluki, РАМШ",S1,"[1.0112338066101074, -0.45509031414985657, 0.2..."
2,"""Азино"" семь-семь-семь По-по-поднял бабла (у) ...",https://pesnihi.com/lyrics/v/vitya-ak/azino-tr...,Витя АК,Азино три топора,"[-0.05332232639193535, -0.2194392830133438, -0..."
3,"""Глаз заглядывает за рамки мира сего в иной И ...",https://pesnihi.com/lyrics/n/namelessjulia/pir...,"namelessjulia, Найтивыход",Пирамида над Кремлём,"[0.0619158111512661, -0.2646079659461975, 0.41..."
4,"""За сином слєді, дорогая"" - шептав мій отєц ум...",https://pesnihi.com/lyrics/h/hamerman-znishhue...,Хамерман Знищує Віруси,Раненое децтво,"[0.9022114276885986, -1.0176315307617188, -0.0..."


In [3]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2", cache_dir=None, do_lower_case=True)
model = BertModel.from_pretrained("pretrained", cache_dir=None)
device="cpu" 
model.to(device)
model.eval()

Some weights of the model checkpoint at pretrained were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at pretrained and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably T

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(83828, 312, padding_idx=0)
    (position_embeddings): Embedding(2048, 312)
    (token_type_embeddings): Embedding(2, 312)
    (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=312, out_features=312, bias=True)
            (key): Linear(in_features=312, out_features=312, bias=True)
            (value): Linear(in_features=312, out_features=312, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=312, out_features=312, bias=True)
            (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [4]:
def fix_errors(text):
  text = text.lower().split("?")[0]
  domain = "https://speller.yandex.net/services/spellservice.json"
  words = text.split()
  words = "+".join(words)
  request = requests.get(domain + "/checkText?text=" + words)
  if request:
    response = [(i["word"], i["s"]) for i in request.json()]
    for word, fixes in response:
      text = text.replace(word, fixes[0])
    return text
  return text

def convert_to_bert_line(text, tokenizer, max_seq_length=2048):
    tokens = "[CLS] " + text
    tokens = tokenizer(tokens, max_length=max_seq_length, padding="max_length", truncation=True)
    return tokens 

def process_request(text, n=5):
  text = text.lower()
  fixed_text = fix_errors(text)
  if fixed_text != text:
    print(f"Do you mean: '{fixed_text}'?")
    if input("Y/n:") == "Y":
      text = fixed_text
  tokens = torch.tensor(convert_to_bert_line(text, tokenizer)["input_ids"], dtype=torch.long).reshape(1, -1)
  preds = model(tokens.to(device),
                attention_mask=(tokens > 0).to(device),
                return_dict=False)
  request_embeds = preds[0].detach().cpu().numpy().tolist()[0][0]
  response = dataset.copy()
  response["dist"] = response["embeds"].progress_apply(lambda x: distance.cosine(request_embeds, x))
  response = response.sort_values(by="dist").head(n)
  return response

def print_response(response):
    wikipedia.set_lang("ru")
    for i in range(response.shape[0]):
        row = response.iloc[i]
        print(f"\nItem #{i + 1}")
        print(f"Author and title: {row['artist']} - {row['title']}")
        print(f"Text sample: {row['text'][:200]}")
        wiki_results = wikipedia.search(row['artist'], results=5)
        wiki_links = []
        for result in wiki_results:
          try:
            wiki_links.append(wikipedia.page(result).url)
          except DisambiguationError:
            wiki_links.append("Too many options to choose")
        print("Wikipedia results:")
        for j in range(len(wiki_links)):
            print(f"#{j + 1} {wiki_results[j]} {wiki_links[j]}")

In [5]:
response = process_request("Москва")
print_response(response)

  0%|          | 0/70217 [00:00<?, ?it/s]


Item #1
Author and title: Маугли - Москва
Text sample: Москва, Москва, Москва, Москва Москва, Москва, Москва (Москва, Москва, Москва Москва, Москва, Москва) Я расту точно плесень на гороскопе Москвы Душат барышень барашы из карманов и малыши Волоку пресно
Wikipedia results:
#1 Маугли https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D1%83%D0%B3%D0%BB%D0%B8
#2 Маугли (мультфильм) https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D1%83%D0%B3%D0%BB%D0%B8_(%D0%BC%D1%83%D0%BB%D1%8C%D1%82%D1%84%D0%B8%D0%BB%D1%8C%D0%BC)
#3 Книга джунглей https://ru.wikipedia.org/wiki/%D0%9A%D0%BD%D0%B8%D0%B3%D0%B0_%D0%B4%D0%B6%D1%83%D0%BD%D0%B3%D0%BB%D0%B5%D0%B9
#4 Маугли (фильм) https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D1%83%D0%B3%D0%BB%D0%B8_(%D1%84%D0%B8%D0%BB%D1%8C%D0%BC)
#5 Маугли (значения) Too many options to choose

Item #2
Author and title: Гуф (GUF), Тимати - Москва
Text sample: Тимати и ГУФ стоят на самом Высом обзорном этаже Москва-сити. Ща поясню О! Москва, в любое время дня и ночи Москва - здесь ты найдёшь

In [6]:
response = process_request("грустная осень")
print_response(response)

  0%|          | 0/70217 [00:00<?, ?it/s]


Item #1
Author and title: Нина Бродская - Август
Text sample: Скоро осень, за окнами август От дождя потемнели кусты И я знаю, что я тебе нравлюсь Как когда-то мне нравился ты Отчего же тоска тебя гложет Отчего ты так грустен со мной Разве в августе сбыться не м
Wikipedia results:
#1 Бродская, Нина Александровна https://ru.wikipedia.org/wiki/%D0%91%D1%80%D0%BE%D0%B4%D1%81%D0%BA%D0%B0%D1%8F,_%D0%9D%D0%B8%D0%BD%D0%B0_%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%BD%D0%B4%D1%80%D0%BE%D0%B2%D0%BD%D0%B0
#2 Бродская Too many options to choose
#3 Список песен на стихи Леонида Дербенёва https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BF%D0%B5%D1%81%D0%B5%D0%BD_%D0%BD%D0%B0_%D1%81%D1%82%D0%B8%D1%85%D0%B8_%D0%9B%D0%B5%D0%BE%D0%BD%D0%B8%D0%B4%D0%B0_%D0%94%D0%B5%D1%80%D0%B1%D0%B5%D0%BD%D1%91%D0%B2%D0%B0
#4 Кристалинская, Майя Владимировна https://ru.wikipedia.org/wiki/%D0%9A%D1%80%D0%B8%D1%81%D1%82%D0%B0%D0%BB%D0%B8%D0%BD%D1%81%D0%BA%D0%B0%D1%8F,_%D0%9C%D0%B0%D0%B9%D1%8F_%D0%92%D

In [7]:
response = process_request("День рожденя")
print_response(response)

Do you mean: 'день рождения'?


  0%|          | 0/70217 [00:00<?, ?it/s]


Item #1
Author and title: Ирина Аллегрова - День рождения
Text sample: Снова, снова праздную я свой день рождения Снова бесшабашную пластинку кручу Только с каждым разом грустнее веселье Я за этот день годом заплачу День рождения, день рождения - Очень трудный день в год
Wikipedia results:
#1 Аллегрова, Ирина Александровна https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D0%BB%D0%B5%D0%B3%D1%80%D0%BE%D0%B2%D0%B0,_%D0%98%D1%80%D0%B8%D0%BD%D0%B0_%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%BD%D0%B4%D1%80%D0%BE%D0%B2%D0%BD%D0%B0
#2 Список песен Ирины Аллегровой https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BF%D0%B5%D1%81%D0%B5%D0%BD_%D0%98%D1%80%D0%B8%D0%BD%D1%8B_%D0%90%D0%BB%D0%BB%D0%B5%D0%B3%D1%80%D0%BE%D0%B2%D0%BE%D0%B9
#3 Электроклуб https://ru.wikipedia.org/wiki/%D0%AD%D0%BB%D0%B5%D0%BA%D1%82%D1%80%D0%BE%D0%BA%D0%BB%D1%83%D0%B1
#4 Рубальская, Лариса Алексеевна https://ru.wikipedia.org/wiki/%D0%A0%D1%83%D0%B1%D0%B0%D0%BB%D1%8C%D1%81%D0%BA%D0%B0%D1%8F,_%D0%9B%D0%B0%D1%80